In [1]:
def readGrid(path):
    with open(f"../assets/data/TennerGrid/problems/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n),  res

if __name__ == "__main__":
    m, n, grid = readGrid("1_6x10")
    print(m, n)
    # print(grid)
    for g in grid:
        print(g)
    

6 10
['-', '4', '-', '2', '-', '9', '3', '8', '7', '5']
['5', '-', '1', '7', '0', '8', '6', '9', '3', '4']
['4', '-', '5', '-', '3', '7', '-', '-', '-', '-']
['2', '6', '4', '8', '5', '-', '-', '7', '-', '-']
['-', '3', '-', '-', '-', '-', '5', '-', '8', '6']
['12', '23', '12', '20', '18', '40', '17', '34', '24', '25']


In [8]:
from z3 import Solver, Int, Distinct, sat, And, Or, If, Sum

def tennerGrid_solver(m, n, grid):
    x = [[Int(f"x_{i}_{j}") for j in range(n)] for i in range(m - 1)]
    s = Solver()

    for i in range(m - 1):
        for j in range(n):
            if grid[i][j].isdigit():
                s.add(x[i][j] == int(grid[i][j]))

    # define the existing number
    cells_constr  = [ And(0 <= x[i][j], x[i][j] <= 9) 
             for i in range(m - 1) for j in range(n) ]
    # define the range of variables
    
    for i in range(m - 1):
        s.add(Distinct([x[i][j] for j in range(n)]))

    for j in range(n):
        s.add(Sum([x[i][j] for i in range(m - 1)]) == int(grid[m - 1][j]))
    # add summation constr ... 
    
    for i in range(m - 1):
        for j in range(n):
            sur = []
            for i1 in range(-1, 2, 1):
                for j1 in range(-1, 2, 1):
                    if (i1 == 0 and j1 == 0) or (i + i1 < 0 or i + i1 >= m - 1) or (j + j1 < 0 or j + j1 >= n) :
                        continue
                    sur.append(x[i + i1][j + j1])
            for k in range(len(sur)):
                s.add(x[i][j] != sur[k])
    
    s.add(cells_constr)
    if s.check() == sat:
        model = s.model()
        r = [ [ model.evaluate(x[i][j]) for j in range(n) ] 
              for i in range(m - 1) ]
        for i in range(m - 1):
            for j in range(n):
                print(r[i][j], end = " ")
            print("")
        for j in range(n):
            print(int(grid[m - 1][j]), end = " ")

if __name__ == "__main__":
    m, n, grid = readGrid("2_6x10")
    tennerGrid_solver(m, n, grid)

4 3 8 7 5 2 9 6 1 0 
7 6 1 3 0 8 5 4 9 2 
0 5 8 6 2 9 3 7 1 4 
3 7 0 1 8 4 2 9 5 6 
9 8 3 7 6 5 0 4 2 1 
23 29 20 24 21 28 19 30 18 13 